# <center> **Requesting an external API**

A partir d'une liste de titres de films nous allons requêter l'API publique [https://www.omdbapi.com](https://www.omdbapi.com)

Nous enregistrerons les données non-structurées (résumé et affiche du film) dans une base NoSQL (MongoDB)

In [226]:
%reset

## **Imports**

In [227]:
import math
import copy
import re
import json
import requests
import numpy as np
import pandas as pd
from tqdm import tqdm
from unidecode import unidecode

# MongoDB / Pymongo
import pymongo
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi
import pprint

pd.set_option('display.max_rows', 10)
tqdm.pandas()

api_key = "b8dd5759"

## **Reading the data**


In [ ]:
df_movies = pd.read_csv('csv/movies_year_2006_to_2010.csv', delimiter = ',', usecols=['title', 'original_title', 'summary', 'url_thumbnail'])
print("Nb movies :", df_movies.shape[0])
df_movies.head(5)

Nb movies : 980


,title,original_title,summary,url_thumbnail
0,Star Wars : Episode III - La Revanche des Sith,Star Wars: Episode III - Revenge of the Sith,La Guerre des Clones fait rage. Une franche ho...,https://fr.web.img6.acsta.net/c_310_420/medias...
1,Le Seigneur des anneaux : le retour du roi,The Lord of the Rings: The Return of the King,"Les armées de Sauron ont attaqué Minas Tirith,...",https://fr.web.img6.acsta.net/c_310_420/medias...
2,Batman Begins,Batman Begins,Comment un homme seul peut-il changer le monde...,https://fr.web.img6.acsta.net/c_310_420/pictur...
3,King Kong,King Kong,"New York, 1933. Ann Darrow est une artiste de ...",https://fr.web.img3.acsta.net/c_310_420/medias...
4,Kill Bill: Volume 1,Kill Bill: Volume 1,Au cours d'une cérémonie de mariage en plein d...,https://fr.web.img6.acsta.net/c_310_420/medias...


In [229]:
def format_string(st):
    ''' format string 
        from "title of the movie" 
        to title+of+the+movie

        Arg: st string to be converted.
    '''
    res = ''
    for c in st:
        if c.isdigit() or c.isalpha() or c.isspace():
            res += unidecode(c)
        else:
            res += ' '
    return '+'.join([word for word in res.split() if len(word) > 1])

def request_omdb_from_title(title):
    ''' Request the omdb API
    
        return a json dictionary with the information about the movie.

        Arg:
         - title: string with title of the movie we want the infos about.
    '''
    url = f"https://www.omdbapi.com/?apikey={api_key}&t={format_string(title)}"
    r = requests.get(url)
    if r.status_code != 200:
        print(f"ERROR {title}, Response Code: {r.status_code}")
        print("Request:", url)
        return {'Response': 'False'}
    return json.loads(r.text)

def get_plot_and_thumbail_from_omdb(title):
    ''' return movie plot and thumbail through an API request.

        return: 
          - plot:      string containing the plot of the movie,
          - thumbnail: string containing the url of the thumbnail.

        Arg: title: string with the title of the movie.
    '''
    plot, thumbnail = '', ''
    res_dict = request_omdb_from_title(title)
    lst_keys = res_dict.keys()
    assert 'Response' in res_dict
    if res_dict['Response'] == 'True':
        # print('res_dict', res_dict)
        assert 'Plot' in lst_keys and 'Poster' in lst_keys
        if res_dict['Plot'] != 'N/A':
            plot = res_dict['Plot']
        if res_dict['Poster'] != '' and res_dict['Poster'] != 'N/A':
            thumbnail = res_dict['Poster']
    return plot + "AND" + thumbnail

## **Get the plot and the thumbnail from the omdb API**

In [230]:
df_movies['temp']      = df_movies['original_title'].apply(get_plot_and_thumbail_from_omdb)
df_movies['plot']      = df_movies['temp'].apply(lambda x : x.split('AND')[0])
df_movies['thumbnail'] = df_movies['temp'].apply(lambda x : x.split('AND')[1])
df_movies['plot']          = np.where(df_movies['plot'] != '', df_movies['plot'], df_movies['summary'])
df_movies['url_thumbnail'] = np.where(df_movies['thumbnail'] != '', df_movies['thumbnail'], df_movies['url_thumbnail'])
df_movies = df_movies[['title', 'original_title', 'plot', 'url_thumbnail']]

In [233]:
df_movies

,title,original_title,plot,url_thumbnail
0,Star Wars : Episode III - La Revanche des Sith,Star Wars: Episode III - Revenge of the Sith,Anakin Skywalker finally makes the transition ...,https://fr.web.img6.acsta.net/c_310_420/medias...
1,Le Seigneur des anneaux : le retour du roi,The Lord of the Rings: The Return of the King,Gandalf and Aragorn lead the World of Men agai...,https://m.media-amazon.com/images/M/MV5BMTZkMj...
2,Batman Begins,Batman Begins,"After witnessing his parents' death, Bruce lea...",https://m.media-amazon.com/images/M/MV5BODIyMD...
3,King Kong,King Kong,A greedy film producer assembles a team of mov...,https://m.media-amazon.com/images/M/MV5BMWY0NW...
4,Kill Bill: Volume 1,Kill Bill: Volume 1,Au cours d'une cérémonie de mariage en plein d...,https://m.media-amazon.com/images/M/MV5BZmRhZG...
...,...,...,...,...
975,Winnie l'ourson et l'Efélant,Pooh's Heffalump Movie,La paisible Forêt des Rêves Bleus est soudain ...,https://fr.web.img2.acsta.net/c_310_420/medias...
976,Bronx à Bel Air,Bringing down the house,When a lonely guy meets a woman on the interne...,https://m.media-amazon.com/images/M/MV5BOTUyMT...
977,Les Enigmes de l'Atlantide (V),Atlantis : Milo's Return,"Milo, Kida et leur équipe vont faire le tour d...",https://fr.web.img3.acsta.net/c_310_420/medias...
978,Le Soleil,Solntse,Ural Airlines flight grounded in cornfield aft...,https://m.media-amazon.com/images/M/MV5BMDQzYT...


## **Store the data in a NoSQL database**

In [234]:
# Connect to MongoDB
client = pymongo.MongoClient("mongodb://localhost:27017/")

# Create database "allocine" (or selects it if already exists)
mydb = client["allocine"]
# client.drop_database("movies")

# Create a collection "movies" (table in SQL)
col_movies = mydb["movies"]
# col_movies.drop()

In [235]:
# Insertion of movie plots in MongoDB database
col_movies.insert_many(df_movies.to_dict(orient='records')) # TO DO ONLY ONCE
# col_movies.drop()

print(client.list_database_names())
print(mydb.list_collection_names())

# for doc in list(col_movies.find().limit(5)):
#     pprint.pprint(doc)

print("Nb documents:", col_movies.count_documents({}))

['Rennes', 'Rennes2', 'admin', 'allocine', 'config', 'local', 'mydatabase', 'test_queries']
['movies']
Nb documents: 5149


## **Checking what is in the Mongo DB**

In [237]:
names = ['csv/movies_year_1960_to_1970.csv',
         'csv/movies_year_1970_to_1980.csv',
         'csv/movies_year_1980_to_1990.csv',
         'csv/movies_year_1990_to_1995.csv',
         'csv/movies_year_1995_to_2000.csv',
         'csv/movies_year_2000_to_2003.csv',
         'csv/movies_year_2003_to_2006.csv',
         'csv/movies_year_2006_to_2010.csv',
         'csv/movies_year_2010_to_2015.csv',
         'csv/movies_year_2015_to_2019.csv',
         'csv/movies_year_2019_to_2022.csv',
         'csv/movies_year_2022_to_2025.csv']

lst_df = [pd.read_csv(name, delimiter = ',', usecols=['title', 'original_title', 'summary', 'url_thumbnail'])
          for name in names]

for df in lst_df:
    print(df.shape[0])

df_csv = pd.concat(lst_df)
print("Nb total de films", df_csv.shape[0])
# df_csv.to_csv('csv/movies_all_years.csv', sep=',', index = False)

# Reading data fmo Mongo DB
client = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = client["allocine"]
col_movies = mydb["movies"]
print("Nb documents:", col_movies.count_documents({}))

# 1960 to 2003 stored in Mongo DB
518 + 678 + 869 + 585 + 807 + 729 +980

518
678
869
585
807
729
980
600
750
800
750
750
Nb total de films 8816
Nb documents: 5149


5166

In [238]:
# Store data in csv file
lst = list(col_movies.find())
df_mongo = pd.DataFrame(lst, columns = ['_id', 'title', 'original_title', 'plot', 'url_thumbnail'])
print(df_mongo.shape[0])
df_mongo.to_csv('csv/mongoDB_1960_to_2006.csv', sep=',', index = False)

5149


**Since we added twice the same 807 movies, we have to remove them from MongoDB**

In [ ]:
client = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = client["allocine"]
col_movies = mydb["movies"]

print("Nb documents:", col_movies.count_documents({}))
col_movies.delete_many({'original_title' : {'$in' : df_movies['original_title'].values.tolist()} })
print("Nb documents:", col_movies.count_documents({}))

Nb documents: 4264
Nb documents: 2639


**Looks like more documents have been deleted**<br>
So we will drop the whole data base and use the csv 'mongoDB.csv' to restore the previous database and re-add the movies needed.

In [ ]:
client = pymongo.MongoClient("mongodb://localhost:27017/")
client.drop_database('allocine')

# Recreate DB
client = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = client["allocine"]
col_movies = mydb["movies"]

## **Query MongoDB Using PyMongo**

### **We create another DB**

In [184]:
client2 = pymongo.MongoClient("mongodb://localhost:27017/")
mydb2 = client2["test_queries"]
# client.drop_database("test_queries")

# Create a collection "movies" (table in SQL)
col_fruits = mydb2["fruits"]
# col_fruits.drop()

col_fruits.insert_many([
        # { "_id": 1, "name": "apples", "qty": 5, "rating": 3, "color": "red", "type": ["fuji", "honeycrisp"] },
        # { "_id": 2, "name": "bananas", "qty": 7, "rating": 4, "color": "yellow", "type": ["cavendish"] },
        { "_id": 3, "name": "oranges", "qty": 6, "rating": 2, "type": ["naval", "mandarin"] },
        { "_id": 4, "name": "pineapple", "qty": 3, "rating": 5, "color": "yellow" },
        { "_id": 5, "name": "test", "qty": 10, "rating": 3, "color": "green", "type": ["fuji", "typeXXX"] },
    ])
# col_fruits.drop()

print(client2.list_database_names())
print(mydb2.list_collection_names())

['Rennes', 'Rennes2', 'admin', 'allocine', 'config', 'local', 'mydatabase', 'test_queries']
['fruits']


**Sources**<br>
https://www.mongodb.com/docs/languages/python/pymongo-driver/current/read/specify-a-query/<br>
https://www.w3resource.com/mongodb/introduction-mongodb.php<br>
https://www.mongodbtutorial.org/mongodb-crud/mongodb-findone/<br>
https://geekflare.com/fr/mongodb-queries-examples/<br>


**Exact Match**

In [175]:
results = col_movies.find({ "title": "Eyes Wide Shut" })
results.to_list()

[{'_id': ObjectId('67b1ae3a1e1d959e842460ed'),
  'title': 'Eyes Wide Shut',
  'original_title': 'Eyes Wide Shut',
  'plot': "A Manhattan doctor embarks on a bizarre, night-long odyssey after his wife's admission of unfulfilled longing.",
  'url_thumbnail': 'https://m.media-amazon.com/images/M/MV5BZTQ0MmM5MDAtYmYyZS00MzlmLTlhZTAtZDJlZWY5ZTZkZjZmXkEyXkFqcGc@._V1_SX300.jpg'},
 {'_id': ObjectId('67b1ae4a1e1d959e84246414'),
  'title': 'Eyes Wide Shut',
  'original_title': 'Eyes Wide Shut',
  'plot': "A Manhattan doctor embarks on a bizarre, night-long odyssey after his wife's admission of unfulfilled longing.",
  'url_thumbnail': 'https://m.media-amazon.com/images/M/MV5BZTQ0MmM5MDAtYmYyZS00MzlmLTlhZTAtZDJlZWY5ZTZkZjZmXkEyXkFqcGc@._V1_SX300.jpg'}]

In [144]:
results = col_fruits.find({ "color": "yellow" })
results.to_list()

[{'_id': 2,
  'name': 'bananas',
  'qty': 7,
  'rating': 4,
  'color': 'yellow',
  'type': ['cavendish']},
 {'_id': 4, 'name': 'pineapple', 'qty': 3, 'rating': 5, 'color': 'yellow'}]

**Comparison operators**

list of operators: https://www.mongodb.com/docs/manual/reference/operator/query-comparison/

In [143]:
results = col_fruits.find({ "rating": { "$gt" : 2 }})
for f in results:
    print(f) 

{'_id': 1, 'name': 'apples', 'qty': 5, 'rating': 3, 'color': 'red', 'type': ['fuji', 'honeycrisp']}
{'_id': 2, 'name': 'bananas', 'qty': 7, 'rating': 4, 'color': 'yellow', 'type': ['cavendish']}
{'_id': 4, 'name': 'pineapple', 'qty': 3, 'rating': 5, 'color': 'yellow'}


**Logical operators**

https://www.mongodb.com/docs/manual/reference/operator/query-logical/

In [147]:
results = col_fruits.find({ 
    "$or": [
        { "qty": { "$gt": 5 }},
        { "color": "yellow" }
    ]
})
for f in results:
    print(f)

{'_id': 2, 'name': 'bananas', 'qty': 7, 'rating': 4, 'color': 'yellow', 'type': ['cavendish']}
{'_id': 3, 'name': 'oranges', 'qty': 6, 'rating': 2, 'type': ['naval', 'mandarin']}
{'_id': 4, 'name': 'pineapple', 'qty': 3, 'rating': 5, 'color': 'yellow'}


**Arrays operators**

https://www.mongodb.com/docs/manual/reference/operator/query-array/

In [158]:
for f in col_fruits.find({}):
    print(f)

print('----')
results = col_fruits.find({
    "type" : { "$size": 2 }
})

for f in results:
    print(f)

{'_id': 1, 'name': 'apples', 'qty': 5, 'rating': 3, 'color': 'red', 'type': ['fuji', 'honeycrisp']}
{'_id': 2, 'name': 'bananas', 'qty': 7, 'rating': 4, 'color': 'yellow', 'type': ['cavendish']}
{'_id': 3, 'name': 'oranges', 'qty': 6, 'rating': 2, 'type': ['naval', 'mandarin']}
{'_id': 4, 'name': 'pineapple', 'qty': 3, 'rating': 5, 'color': 'yellow'}
{'_id': 5, 'name': 'test', 'qty': 10, 'rating': 3, 'color': 'green', 'type': ['fuji', 'typeXXX']}
----
{'_id': 1, 'name': 'apples', 'qty': 5, 'rating': 3, 'color': 'red', 'type': ['fuji', 'honeycrisp']}
{'_id': 3, 'name': 'oranges', 'qty': 6, 'rating': 2, 'type': ['naval', 'mandarin']}
{'_id': 5, 'name': 'test', 'qty': 10, 'rating': 3, 'color': 'green', 'type': ['fuji', 'typeXXX']}


In [159]:
results = col_fruits.find({
    "type" : { "$all": ['fuji'] }
})

for f in results:
    print(f)

{'_id': 1, 'name': 'apples', 'qty': 5, 'rating': 3, 'color': 'red', 'type': ['fuji', 'honeycrisp']}
{'_id': 5, 'name': 'test', 'qty': 10, 'rating': 3, 'color': 'green', 'type': ['fuji', 'typeXXX']}


**Element Operator**

to check if a field exists

In [160]:
results = col_fruits.find( { "color" : { "$exists": "true" }} )
for f in results:
    print(f)

{'_id': 1, 'name': 'apples', 'qty': 5, 'rating': 3, 'color': 'red', 'type': ['fuji', 'honeycrisp']}
{'_id': 2, 'name': 'bananas', 'qty': 7, 'rating': 4, 'color': 'yellow', 'type': ['cavendish']}
{'_id': 4, 'name': 'pineapple', 'qty': 3, 'rating': 5, 'color': 'yellow'}
{'_id': 5, 'name': 'test', 'qty': 10, 'rating': 3, 'color': 'green', 'type': ['fuji', 'typeXXX']}


**Evaluation operators**

https://www.mongodb.com/docs/manual/reference/operator/query-evaluation/

In [ ]:
results = col_fruits.find({ "name" : { "$regex" : "p{2,}" }} ) ## At least 2 consecutives p
for f in results:
    print(f)

{'_id': 1, 'name': 'apples', 'qty': 5, 'rating': 3, 'color': 'red', 'type': ['fuji', 'honeycrisp']}
{'_id': 4, 'name': 'pineapple', 'qty': 3, 'rating': 5, 'color': 'yellow'}


**Delete data**

In [ ]:
# Delete_one
for f in col_fruits.find({}):
    print(f)

col_fruits.delete_one({'_id' : 4})

for f in col_fruits.find({}):
    print(f)

{'_id': 1, 'name': 'apples', 'qty': 5, 'rating': 3, 'color': 'red', 'type': ['fuji', 'honeycrisp']}
{'_id': 2, 'name': 'bananas', 'qty': 7, 'rating': 4, 'color': 'yellow', 'type': ['cavendish']}
{'_id': 3, 'name': 'oranges', 'qty': 6, 'rating': 2, 'type': ['naval', 'mandarin']}
{'_id': 5, 'name': 'test', 'qty': 10, 'rating': 3, 'color': 'green', 'type': ['fuji', 'typeXXX']}
{'_id': 4, 'name': 'pineapple', 'qty': 3, 'rating': 5, 'color': 'yellow'}
{'_id': 1, 'name': 'apples', 'qty': 5, 'rating': 3, 'color': 'red', 'type': ['fuji', 'honeycrisp']}
{'_id': 2, 'name': 'bananas', 'qty': 7, 'rating': 4, 'color': 'yellow', 'type': ['cavendish']}
{'_id': 3, 'name': 'oranges', 'qty': 6, 'rating': 2, 'type': ['naval', 'mandarin']}
{'_id': 5, 'name': 'test', 'qty': 10, 'rating': 3, 'color': 'green', 'type': ['fuji', 'typeXXX']}
{'_id': 4, 'name': 'pineapple', 'qty': 3, 'rating': 5, 'color': 'yellow'}


In [ ]:
# Delete_many
for f in col_fruits.find({}):
    print(f)

col_fruits.delete_many({'_id' : {'$gte' : 3} })

for f in col_fruits.find({}):
    print(f)

{'_id': 1, 'name': 'apples', 'qty': 5, 'rating': 3, 'color': 'red', 'type': ['fuji', 'honeycrisp']}
{'_id': 2, 'name': 'bananas', 'qty': 7, 'rating': 4, 'color': 'yellow', 'type': ['cavendish']}
{'_id': 3, 'name': 'oranges', 'qty': 6, 'rating': 2, 'type': ['naval', 'mandarin']}
{'_id': 4, 'name': 'pineapple', 'qty': 3, 'rating': 5, 'color': 'yellow'}
{'_id': 5, 'name': 'test', 'qty': 10, 'rating': 3, 'color': 'green', 'type': ['fuji', 'typeXXX']}
{'_id': 1, 'name': 'apples', 'qty': 5, 'rating': 3, 'color': 'red', 'type': ['fuji', 'honeycrisp']}
{'_id': 2, 'name': 'bananas', 'qty': 7, 'rating': 4, 'color': 'yellow', 'type': ['cavendish']}
{'_id': 3, 'name': 'oranges', 'qty': 6, 'rating': 2, 'type': ['naval', 'mandarin']}
{'_id': 4, 'name': 'pineapple', 'qty': 3, 'rating': 5, 'color': 'yellow'}
{'_id': 5, 'name': 'test', 'qty': 10, 'rating': 3, 'color': 'green', 'type': ['fuji', 'typeXXX']}


In [186]:
# Delete_many
for f in col_fruits.find({}):
    print(f)

col_fruits.delete_many({'_id' : {'$in' : [1, 3, 5]} })

for f in col_fruits.find({}):
    print(f)

{'_id': 1, 'name': 'apples', 'qty': 5, 'rating': 3, 'color': 'red', 'type': ['fuji', 'honeycrisp']}
{'_id': 2, 'name': 'bananas', 'qty': 7, 'rating': 4, 'color': 'yellow', 'type': ['cavendish']}
{'_id': 3, 'name': 'oranges', 'qty': 6, 'rating': 2, 'type': ['naval', 'mandarin']}
{'_id': 4, 'name': 'pineapple', 'qty': 3, 'rating': 5, 'color': 'yellow'}
{'_id': 5, 'name': 'test', 'qty': 10, 'rating': 3, 'color': 'green', 'type': ['fuji', 'typeXXX']}
{'_id': 2, 'name': 'bananas', 'qty': 7, 'rating': 4, 'color': 'yellow', 'type': ['cavendish']}
{'_id': 4, 'name': 'pineapple', 'qty': 3, 'rating': 5, 'color': 'yellow'}
